In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense
import keras
import tensorflow as tf
import os
os.getcwd()
os.chdir("/Volumes/Rohith/ipl-iit/")

Using TensorFlow backend.


In [2]:
data = pd.read_csv('all_matches.csv')

/Users/rohith/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.drop(columns=["match_id", "season","start_date"], inplace=True)

In [4]:
data_n1 = data.to_numpy()

In [5]:
players = np.append(np.unique(np.append(np.unique(np.append(np.unique(data['striker']),np.unique(data['non_striker']))),np.unique(data['bowler']))),"Shahrukh Khan")

In [6]:
def create_train_data(data_n):
    T = [[]]
    for i in data_n:
        t = []
        t.extend([i[0], i[1], i[3], i[4], [], []])
        if(float(i[2]) < 6.0):
            if(i[5] not in t[4]):
                t[4].extend([i[5]])
            if(i[6] not in t[4]):
                t[4].extend([i[6]])
            if(i[7] not in t[5]):
                t[5].extend([i[7]])

            if(t[0] not in T[-1]):
                T.append(t)
    #         if(t not in T):
    #             T.append(t)


            if(t[0] in T[-1]):
                if(t[1] not in T[-1]):
                    T.append(t)

                if(t[1] in T[-1]):
                    for k in range(len(t[4])):
                        if(t[4][k] not in T[-1][4]):
                            T[-1][4].extend([t[4][k]])
                    for k in range(len(t[5])):
                        if(t[5][k] not in T[-1][5]):
                            T[-1][5].extend([t[5][k]])
    T.pop(0)
    
    # mearging bats and bowls
   # for i in range(len(T)):
    #    T[i][4].extend(T[i][5])
     #   T[i].pop(5)
    
    # Mearging bowlers and bats with total
   # for i in range(len(T)):
    #    T[i].extend(T[i][4])
     #   T[i].pop(4)
    return T

In [7]:
input_data = create_train_data(data_n1)

In [8]:
def score_list(ddd1):
    sum_ = ddd1[0][8]+ddd1[0][9]
    yy = []
    for i in range(1, len(ddd1)):
        if(int(ddd1[i][2])<6.0):
            if(ddd1[i][1] == ddd1[i-1][1] and ddd1[i][0] == ddd1[i-1][0]):
                sum_ += ddd1[i][8]+ddd1[i][9]

            elif(ddd1[i][1] != ddd1[i-1][1] and ddd1[i][0] != ddd1[i-1][0]):
                yy.append(sum_)
                sum_ = ddd1[i][8]+ddd1[i][9]

            elif(ddd1[i][1] != ddd1[i-1][1] and ddd1[i][0] == ddd1[i-1][0]):
                yy.append(sum_)
                sum_ = ddd1[i][8]+ddd1[i][9]

            elif(ddd1[i][1] == ddd1[i-1][1] and ddd1[i][0] != ddd1[i-1][0]):
                yy.append(sum_)
                sum_ = ddd1[i][8]+ddd1[i][9]
            
            else:
                print(ddd1[i])
         
#         if(i == len(ddd1)-1):
#             yy.append(sum_)
#   print()         sum_ = 0
    yy.append(sum_)
    return yy

In [9]:
y = np.array(score_list(data_n1))

In [10]:
ylist = y.tolist()

In [11]:
ylist.extend([26,32,46,45,55,39,32,55,54,45,32,59,21,45,25,50,51,65,51,56,37,42,36,43,39,50,47,49,45,67,49,36,49,58,42,57,39,63])
y = np.array(ylist)

In [12]:
max_bat=0
max_bowl=0
for i in input_data:
    if len(i[4])>max_bat:
        max_bat = len(i[4])
    if len(i[5])>max_bowl:
        max_bowl = len(i[5])

In [13]:
len(y)

1702

In [14]:
for i in input_data:
    for j in range(max_bat-len(i[4])):
        i[4].extend(["_"])
    for j in range(max_bowl-len(i[5])):
        i[5].extend(["_"])

In [15]:
T = input_data

In [16]:
import glob
path = "Archive/" # use your path
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
frame = pd.concat(li, axis=0, ignore_index=True)

In [17]:
input_1 = frame.values.tolist()
for i in input_1:
    i[4] = i[4].split(",")
    i[5] = i[5].split(",")
for i in input_1:
    for j in range(max_bat-len(i[4])):
        i[4].extend(["_"])
    for j in range(max_bowl-len(i[5])):
        i[5].extend(["_"])

In [18]:
T.extend(input_1[::-1])

In [19]:
 # mearging bats and bowls
for i in range(len(T)):
        T[i][4].extend(T[i][5])
        T[i].pop(5)
    
    #Mearging bowlers and bats with total
for i in range(len(T)):
    T[i].extend(T[i][4])
    T[i].pop(4)

In [20]:
X = pd.DataFrame(T)

In [21]:
X.tail(-15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
15,"MA Chidambaram Stadium, Chepauk",2,Mumbai Indians,Chennai Super Kings,L Ronchi,ST Jayasuriya,RV Uthappa,SM Pollock,_,_,_,_,_,JDP Oram,MS Gony,_,_,_,_
16,"Rajiv Gandhi International Stadium, Uppal",1,Deccan Chargers,Rajasthan Royals,AC Gilchrist,VVS Laxman,Shahid Afridi,A Symonds,_,_,_,_,_,MM Patel,SR Watson,YK Pathan,_,_,_
17,"Rajiv Gandhi International Stadium, Uppal",2,Rajasthan Royals,Deccan Chargers,Kamran Akmal,GC Smith,YK Pathan,_,_,_,_,_,_,WPUJC Vaas,RP Singh,PP Ojha,_,_,_
18,"Punjab Cricket Association Stadium, Mohali",1,Kings XI Punjab,Mumbai Indians,K Goel,IK Pathan,KC Sangakkara,_,_,_,_,_,_,SM Pollock,A Nehra,DS Kulkarni,_,_,_
19,"Punjab Cricket Association Stadium, Mohali",2,Mumbai Indians,Kings XI Punjab,L Ronchi,ST Jayasuriya,RV Uthappa,DJ Bravo,_,_,_,_,_,B Lee,IK Pathan,VRV Singh,S Sreesanth,_,_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,Narendra Modi Stadium,2,Kolkata Knight Riders,Punjab Kings,N Rana,Shubman Gill,RA Tripathi,SP Narine,EJG Morgan,_,_,_,_,Mohammed Shami,MC Henriques,Arshdeep Singh,_,_,_
1698,Narendra Modi Stadium,1,Royal Challengers Bangalore,Delhi Capitals,D Padikkal,V Kohli,GJ Maxwell,RM Patidar,_,_,_,_,_,K Rabada,I Sharma,Avesh Khan,_,_,_
1699,Narendra Modi Stadium,2,Delhi Capitals,Royal Challengers Bangalore,PP Shaw,S Dhawan,SPD Smith,RR Pant,_,_,_,_,_,DR Sams,Mohammed Siraj,Kyle Jamieson,_,_,_
1700,Arun Jaitley Stadium,1,Sunrisers Hyderabad,Chennai Super Kings,DA Warner,JM Bairstow,MK Pandey,_,_,_,_,_,_,DL Chahar,SM Curran,_,_,_,_


In [22]:
X.to_csv("X.csv")

In [23]:
pd.DataFrame(y).to_csv("y.csv")

In [24]:
XY = X.copy()
XY["Score"] = y

In [25]:
XY.to_csv("XV.csv")

In [26]:
from sklearn import preprocessing
#le_stadium = preprocessing.LabelEncoder()
#le_stadium.fit(X[0])
#X[0] = le_stadium.transform(X[0])

le_teams = preprocessing.LabelEncoder()
le_teams.fit(X[2])
X[2] = le_teams.transform(X[2])
X[3] = le_teams.transform(X[3])

In [27]:
X_modified = []
for i in range(4,X.shape[1]):
    X_modified.extend(X[i])

In [28]:
for i in X_modified:
    if(i == None):
        print(i)

In [29]:
p = players.tolist()
p.extend(["Jhye Richardson","Chetan Sakariya","Fabian Allen",'Riley Meredith','D Malan',"_"])
players = np.array(p)

In [30]:
players.shape[0]

590

In [31]:
le_players = preprocessing.LabelEncoder().fit(players)

In [32]:
X.replace("F Du Plessis","F du Plessis",inplace=True)
X.replace("J Buttler","JC Buttler",inplace=True)
X.replace("P Cummins","PJ Cummins",inplace=True)
X.replace("Kyle Jamieson","KA Jamieson",inplace=True)

In [33]:
for i in range(4,X.shape[1]):
    X[i] = le_players.transform(X[i])

In [34]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,M Chinnaswamy Stadium,1,7,14,474,83,435,589,589,589,589,589,589,369,588,21,589,589,589
1,M Chinnaswamy Stadium,2,14,7,398,569,557,217,114,334,589,589,589,24,193,22,589,589,589
2,"Punjab Cricket Association Stadium, Mohali",1,0,5,377,322,309,589,589,589,589,589,589,77,462,231,589,589,589
3,"Punjab Cricket Association Stadium, Mohali",2,5,0,237,231,248,589,589,589,589,589,589,213,332,589,589,589,589
4,Feroz Shah Kotla,1,11,3,533,582,501,287,589,589,589,589,589,172,74,310,589,589,589


In [35]:
le_stad = {'Arun Jaitley Stadium':0, 'Barabati Stadium':1, 'Brabourne Stadium':2,
       'Buffalo Park':3, 'De Beers Diamond Oval':4,
       'Dr DY Patil Sports Academy':5,
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium':6,
       'Dubai International Cricket Stadium':7, 'Eden Gardens':8,
       'Feroz Shah Kotla':9, 'Green Park':10,
       'Himachal Pradesh Cricket Association Stadium':11,
       'Holkar Cricket Stadium':12, 'JSCA International Stadium Complex':13,
       'Kingsmead':14, 'M Chinnaswamy Stadium':15, 'M.Chinnaswamy Stadium':15,
       'MA Chidambaram Stadium':16, 'MA Chidambaram Stadium, Chepauk':16,
       'MA Chidambaram Stadium, Chepauk, Chennai':16,
       'Maharashtra Cricket Association Stadium':17, 'Nehru Stadium':18,
       'New Wanderers Stadium':19, 'Newlands':20, 'OUTsurance Oval':21,
       'Punjab Cricket Association IS Bindra Stadium':22,
       'Punjab Cricket Association IS Bindra Stadium, Mohali':22,
       'Punjab Cricket Association Stadium, Mohali':22,
       'Rajiv Gandhi International Stadium':23,
       'Rajiv Gandhi International Stadium, Uppal':23,
       'Sardar Patel Stadium, Motera':24,
       'Saurashtra Cricket Association Stadium':25, 'Sawai Mansingh Stadium':26,
       'Shaheed Veer Narayan Singh International Stadium':27,
       'Sharjah Cricket Stadium':28, 'Sheikh Zayed Stadium':29,
       "St George's Park":30, 'Subrata Roy Sahara Stadium':31,
       'SuperSport Park':32, 'Vidarbha Cricket Association Stadium, Jamtha':33,
       'Wankhede Stadium':34, 'Wankhede Stadium, Mumbai':34,'Narendra Modi Stadium':35,'Arun Jaitley Stadium':36}

In [36]:
X_s = []
for i in X[0]:
    X_s.append(le_stad[i])

In [37]:
X[0] = X_s

In [38]:
X.replace(players.shape[0]-1,-1,inplace=True)

In [39]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,15,1,7,14,474,83,435,-1,-1,-1,-1,-1,-1,369,588,21,-1,-1,-1
1,15,2,14,7,398,569,557,217,114,334,-1,-1,-1,24,193,22,-1,-1,-1
2,22,1,0,5,377,322,309,-1,-1,-1,-1,-1,-1,77,462,231,-1,-1,-1
3,22,2,5,0,237,231,248,-1,-1,-1,-1,-1,-1,213,332,-1,-1,-1,-1
4,9,1,11,3,533,582,501,287,-1,-1,-1,-1,-1,172,74,310,-1,-1,-1


In [40]:
import pickle
scalerfile = 'DataScaler.sav'
pickle.dump(xScaled, open(scalerfile, 'wb'))

NameError: name 'xScaled' is not defined

In [41]:
scalerfile = 'le_stadium.pkl'
pickle.dump(le_stad, open(scalerfile, 'wb'))

In [42]:
#np.save('le_stadium.npy', le_stad)
np.save('le_teams.npy', le_teams.classes_)
np.save('le_players.npy', le_players.classes_)

In [43]:
y = np.array(y)
my = max(y)
y1 = y/my

In [44]:
my

105

In [45]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, test_size=0.1)

In [46]:
Xtrain.shape

(1531, 19)

In [47]:
ytrain.shape

(1531,)

In [48]:
XY = X.copy()
XY["Score"] = y
XY.to_csv("XY_coded.csv")

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(69, kernel_initializer='normal',input_dim = Xtrain.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(69, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(69, kernel_initializer='normal',activation='relu'))


# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))
optimizer = keras.optimizers.Adam(lr=0.001, decay=5e-4)
# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
NN_model.fit(X,np.array(y))

In [ ]:
test_eval = NN_model.evaluate(Xtest,np.array(ytest), verbose=0)

In [ ]:
ypred = NN_model.predict(Xtest)
ypred = [int(i) for i in np.array(ypred)]
ytest_1 = [int(i) for i in ytest]

In [53]:
from sklearn.metrics import r2_score
m = np.sum((np.array(ytest_1)-np.array(ypred))**2)
print(m)
r2_score(ytest_1,ypred)

15493


0.42414893991619484

In [189]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(criterion='friedman_mse',max_depth=150,n_estimators=150, min_samples_leaf=1,random_state=69,min_samples_split=2,min_weight_fraction_leaf=0.000001)
regr.fit(X,y)
ypred = regr.predict(Xtest)
ypred = [int(i) for i in np.array(ypred)]
ytest_1 = [int(i) for i in ytest]
m = np.sum((np.array(ytest_1)-np.array(ypred))**2)
print(m)
r2_score(ytest_1,ypred)

2523


0.9062239576201226

In [ ]:
m = np.sum((np.array(ytest_1)-np.array(ypred))**2)

In [ ]:
np.mean(np.array(ytest_1)/np.array(ypred))

In [ ]:
def r2_scorer(clf, X, y):
    y_pred = clf.predict(X)
    y_1 = np.array(y_pred)
    y_1 = [int(i) for i in y_1]
    y = [int(i) for i in np.array(y)]
    m = np.sum((np.array(y_1)-np.array(y))**2)
    return m

In [ ]:
filename = 'RF_trail_11.sav'
pickle.dump(regr, open(filename, 'wb'))

In [ ]:
le_teams.classes_

In [ ]:
le_players.classes_

In [50]:
from sklearn.ensemble import RandomForestRegressor
R = RandomForestRegressor(
    criterion="friedman_mse",
    max_depth=5,
    max_features=0.9999999992016524,
    min_samples_leaf=4,
    min_samples_split=3,
    n_estimators=52,
    n_jobs=2,
    random_state=33,
)

In [60]:
R.fit(Xtrain,ytrain)

RandomForestRegressor(criterion='friedman_mse', max_depth=5,
                      max_features=0.9999999992016524, min_samples_leaf=4,
                      min_samples_split=3, n_estimators=52, n_jobs=2,
                      random_state=33)

In [61]:
ypred = R.predict(Xtest)
ypred = [int(i) for i in np.array(ypred)]
ytest_1 = [int(i) for i in ytest]
m = np.sum((np.array(ytest_1)-np.array(ypred))**2)
print(m)
r2_score(ytest_1,ypred)

17461


0.35100139675186726

In [200]:
from sklearn.ensemble import GradientBoostingRegressor as GBR
G = GBR(max_depth=100,n_estimators=100, min_samples_leaf=3,random_state=69,min_samples_split=2)
G.fit(X,y)

GradientBoostingRegressor(max_depth=100, min_samples_leaf=3, random_state=69)

In [201]:
ypred = G.predict(Xtest)
ypred_1 = [int(i) for i in np.array(ypred)]
ytest_1 = [int(i) for i in ytest]
m = np.sum((np.array(ytest_1)-np.array(ypred_1))**2)
print(m)

87


In [107]:
r2_score(ytest_1,ypred_1)

0.9967663433662112

In [197]:
from sklearn.ensemble import ExtraTreesRegressor as ER
E = ER(max_depth=150,n_estimators=150, min_samples_leaf=1,random_state=69,min_samples_split=0.3)
E.fit(X,y)
ypred = E.predict(Xtest)
ypred_1 = [int(i) for i in np.array(ypred)]
ytest_1 = [int(i) for i in ytest]
m = np.sum((np.array(ytest_1)-np.array(ypred))**2)
print(m)

17351.845350491785


In [121]:
filename = 'G_trial_1.sav'
pickle.dump(G, open(filename, 'wb'))

In [135]:
filename = 'E_trial_2.sav'
pickle.dump(E, open(filename, 'wb'))

In [163]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
clf = HistGradientBoostingRegressor(max_depth=50, min_samples_leaf=2,random_state=69,scoring="mse").fit(Xtrain, ytrain)

In [164]:
ypred = clf.predict(Xtest)
ypred_1 = [int(i) for i in np.array(ypred)]
ytest_1 = [int(i) for i in ytest]
m = np.sum((np.array(ytest_1)-np.array(ypred_1))**2)
print(m)

18935


In [181]:
from sklearn.ensemble import AdaBoostRegressor
reg = AdaBoostRegressor(loss="square",learning_rate=0.0000000000000000001,n_estimators=150).fit(X,y)

In [182]:
ypred = reg.predict(Xtest)
ypred_1 = [int(i) for i in np.array(ypred)]
ytest_1 = [int(i) for i in ytest]
m = np.sum((np.array(ytest_1)-np.array(ypred_1))**2)
print(m)

17689
